## Real-world examples
#### Print the return type
You are debugging a package that you've been working on with your friends. Something weird is happening with the data being returned from one of your functions, but you're not even sure which function is causing the trouble. You know that sometimes bugs can sneak into your code when you are expecting a function to return one thing, and it returns something different. For instance, if you expect a function to return a numpy array, but it returns a list, you can get unexpected behavior. To ensure this is not what is causing the trouble, you decide to write a decorator, print_return_type(), that will print out the type of the variable that gets returned from every call of any function it is decorating.

In [1]:
def print_return_type(func):
  # Define wrapper(), the decorated function
  def wrapper(*args, **kwargs):
    # Call the function being decorated
    result = func(*args, **kwargs)
    print('{}() returned type {}'.format(
      func.__name__, type(result)
    ))
    return result
  # Return the decorated function
  return wrapper
  
@print_return_type
def foo(value):
  return value
  
print(foo(42))
print(foo([1, 2, 3]))
print(foo({'a': 42}))

foo() returned type <class 'int'>
42
foo() returned type <class 'list'>
[1, 2, 3]
foo() returned type <class 'dict'>
{'a': 42}


Righteous return types! Your new decorator helps you examine the results of your functions at runtime. Now you can apply this decorator to every function in the package you are developing and run your scripts. Being able to examine the types of your return values will help you understand what is happening and will hopefully help you find the bug.

#### Counter
You're working on a new web app, and you are curious about how many times each of the functions in it gets called. So you decide to write a decorator that adds a counter to each function that you decorate. You could use this information in the future to determine whether there are sections of code that you could remove because they are no longer being used by the app.

In [2]:
def counter(func):
  def wrapper(*args, **kwargs):
    wrapper.count += 1
    # Call the function being decorated and return the result
    return wrapper
  wrapper.count = 0
  # Return the new decorated function
  return wrapper

# Decorate foo() with the counter() decorator
@counter
def foo():
  print('calling foo()')
  
foo()
foo()

print('foo() was called {} times.'.format(foo.count))

foo() was called 2 times.


Cool counting! Now you can go decorate a bunch of functions with the counter() decorator, let your program run for a while, and then print out how many times each function was called.

It seems a little magical that you can reference the wrapper() function from _inside_ the definition of wrapper() as we do here on line 3. That's just one of the many neat things about functions in Python -- any function, not just decorators.

## Decorators and metadata
#### Preserving docstrings when decorating functions
Your friend has come to you with a problem. They've written some nifty decorators and added them to the functions in the open-source library they've been working on. However, they were running some tests and discovered that all of the docstrings have mysteriously disappeared from their decorated functions. Show your friend how to preserve docstrings and other metadata when writing decorators.

In [3]:
def add_hello(func):
  def wrapper(*args, **kwargs):
    print('Hello')
    return func(*args, **kwargs)
  return wrapper

# Decorate print_sum() with the add_hello() decorator
@add_hello
def print_sum(a, b):
  """Adds two numbers and prints the sum"""
  print(a + b)
  
print_sum(10, 20)
print_sum_docstring = print_sum.__doc__
print(print_sum_docstring)

Hello
30
None


In [4]:
def add_hello(func):
  # Add a docstring to wrapper
  def wrapper(*args, **kwargs):
    """Print 'hello' and then call the decorated function."""
    print('Hello')
    return func(*args, **kwargs)
  return wrapper

@add_hello
def print_sum(a, b):
  """Adds two numbers and prints the sum"""
  print(a + b)
  
print_sum(10, 20)
print_sum_docstring = print_sum.__doc__
print(print_sum_docstring)

Hello
30
Print 'hello' and then call the decorated function.


In [5]:
# Import the function you need to fix the problem
from functools import wraps

def add_hello(func):
  def wrapper(*args, **kwargs):
    """Print 'hello' and then call the decorated function."""
    print('Hello')
    return func(*args, **kwargs)
  return wrapper
  
@add_hello
def print_sum(a, b):
  """Adds two numbers and prints the sum"""
  print(a + b)
  
print_sum(10, 20)
print_sum_docstring = print_sum.__doc__
print(print_sum_docstring)

Hello
30
Print 'hello' and then call the decorated function.


In [6]:
from functools import wraps

def add_hello(func):
  # Decorate wrapper() so that it keeps func()'s metadata
  @wraps(func)
  def wrapper(*args, **kwargs):
    """Print 'hello' and then call the decorated function."""
    print('Hello')
    return func(*args, **kwargs)
  return wrapper
  
@add_hello
def print_sum(a, b):
  """Adds two numbers and prints the sum"""
  print(a + b)
  
print_sum(10, 20)
print_sum_docstring = print_sum.__doc__
print(print_sum_docstring)

Hello
30
Adds two numbers and prints the sum


That's a wrap! Your friend was concerned that they couldn't print the docstrings of their functions. They now realize that the strange behavior they were seeing was caused by the fact that they were accidentally printing the wrapper() docstring instead of the docstring of the original function. After adding @wraps(func) to all of their decorators, they see that the docstrings are back where they expect them to be.

#### Measuring decorator overhead
Your boss wrote a decorator called check_everything() that they think is amazing, and they are insisting you use it on your function. However, you've noticed that when you use it to decorate your functions, it makes them run much slower. You need to convince your boss that the decorator is adding too much processing time to your function. To do this, you are going to measure how long the decorated function takes to run and compare it to how long the undecorated function would have taken to run. This is the decorator in question:<br>
<br>
def check_everything(func):<br>
  @wraps(func)<br>
  def wrapper(*args, **kwargs):<br>
    check_inputs(*args, **kwargs)<br>
    result = func(*args, **kwargs)<br>
    check_outputs(result)<br>
    return result<br>
  return wrapper<br>

In [7]:
@check_everything
def duplicate(my_list):
  """Return a new list that repeats the input twice"""
  return my_list + my_list

t_start = time.time()
duplicated_list = duplicate(list(range(50)))
t_end = time.time()
decorated_time = t_end - t_start

t_start = time.time()
# Call the original function instead of the decorated one
duplicated_list = duplicate.__wrapped__(list(range(50)))
t_end = time.time()
undecorated_time = t_end - t_start

print('Decorated time: {:.5f}s'.format(decorated_time))
print('Undecorated time: {:.5f}s'.format(undecorated_time))

NameError: name 'check_everything' is not defined

Wow! Your function ran approximately 10,000 times faster without your boss's decorator. At least they were smart enough to add @wraps(func) to the nested wrapper() function so that you were able to access the original function. You should show them the results of this test. Be sure to ask for a raise while you're at it!

## Decorators that take arguments

In [9]:
def run_n_times(n):
  """Define and return a decorator"""
  def decorator(func):
    def wrapper(*args, **kwargs):
      for i in range(n):
        func(*args, **kwargs)
    return wrapper
  return decorator

# Make print_sum() run 10 times with the run_n_times() decorator
@run_n_times(10)
def print_sum(a, b):
  print(a + b)
  
print_sum(15, 20)

35
35
35
35
35
35
35
35
35
35


In [10]:
# Use run_n_times() to create the run_five_times() decorator
run_five_times = run_n_times(5)

@run_five_times
def print_sum(a, b):
  print(a + b)
  
print_sum(4, 100)

104
104
104
104
104


In [11]:
# Modify the print() function to always run 20 times
print = run_n_times(20)(print)

print('What is happening?!?!')

What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!
What is happening?!?!


Good job!
Good job!
Good job!

You've become an expert at using decorators. Notice how when you use decorator syntax for a decorator that takes arguments, you need to call the decorator by adding parentheses, but you don't add parenthesis for decorators that don't take arguments.

_Warning: overwriting commonly used functions is probably not a great idea, so think twice before using these powers for evil._

#### HTML Generator
You are writing a script that generates HTML for a webpage on the fly. So far, you have written two decorators that will add bold or italics tags to any function that returns a string. You notice, however, that these two decorators look very similar. Instead of writing a bunch of other similar looking decorators, you want to create one decorator, html(), that can take any pair of opening and closing tags.<br>
<br>
def bold(func):<br>
  @wraps(func)<br>
  def wrapper(*args, **kwargs):<br>
    msg = func(*args, **kwargs)<br>
    return '<b>{}</b>'.format(msg)<br>
  return wrapper<br>
def italics(func):<br>
  @wraps(func)<br>
  def wrapper(*args, **kwargs):<br>
    msg = func(*args, **kwargs)<br>
    return '<i>{}</i>'.format(msg)<br>
  return wrapper<br>

In [12]:
def html(open_tag, close_tag):
  def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
      msg = func(*args, **kwargs)
      return '{}{}{}'.format(open_tag, msg, close_tag)
    # Return the decorated function
    return wrapper
  # Return the decorator
  return decorator

In [13]:
# Make hello() return bolded text
@html('<b>', '</b>')
def hello(name):
  return 'Hello {}!'.format(name)

print(hello('Alice'))

<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>
<b>Hello Alice!</b>


In [14]:
# Make goodbye() return italicized text
@html('<i>', '</i>')
def goodbye(name):
  return 'Goodbye {}.'.format(name)
  
print(goodbye('Alice'))

<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>
<i>Goodbye Alice.</i>


In [15]:
# Wrap the result of hello_goodbye() in <div> and </div>
@html('<div>', '</div>')
def hello_goodbye(name):
  return '\n{}\n{}\n'.format(hello(name), goodbye(name))
  
print(hello_goodbye('Alice'))

<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>
<div>
<b>H

That's some HTML hotness! With the new html() decorator you can focus on writing simple functions that return the information you want to display on the webpage and let the decorator take care of wrapping them in the appropriate HTML tags.

## Timeout(): a real world example
#### Tag your functions
Tagging something means that you have given that thing one or more strings that act as labels. For instance, we often tag emails or photos so that we can search for them later. You've decided to write a decorator that will let you tag your functions with an arbitrary list of tags. You could use these tags for many things:

Adding information about who has worked on the function, so a user can look up who to ask if they run into trouble using it.
Labeling functions as "experimental" so that users know that the inputs and outputs might change in the future.
Marking any functions that you plan to remove in a future version of the code.
Etc.

In [16]:
def tag(*tags):
  # Define a new decorator, named "decorator", to return
  def decorator(func):
    # Ensure the decorated function keeps its metadata
    @wraps(func)
    def wrapper(*args, **kwargs):
      # Call the function being decorated and return the result
      return func(*args, **kwargs)
    wrapper.tags = tags
    return wrapper
  # Return the new decorator
  return decorator

@tag('test', 'this is a tag')
def foo():
  pass

print(foo.tags)

('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')
('test', 'this is a tag')


Terrific tagging! With this new decorator, you can do some really interesting things. For instance, you could tag a bunch of image transforming functions, and then write code that searches for all of the functions that transform images and apply them, one after the other, on a given input image. What other neat uses can you come up with for this decorator?

#### Check the return type
Python's flexibility around data types is usually cited as one of the benefits of the language. It can sometimes cause problems though if incorrect data types go unnoticed. You've decided that in order to ensure your code is doing exactly what you want it to do, you will explicitly check the return types in all of your functions and make sure they're returning what you expect. To do that, you are going to create a decorator that checks if the return type of the decorated function is correct.

Note: assert is a keyword that you can use to test whether something is true. If you type assert condition and condition is True, this function doesn't do anything. If condition is False, this function raises an error. The type of error that it raises is called an AssertionError.

In [17]:
def returns_dict(func):
  # Complete the returns_dict() decorator
  def wrapper(*args, **kwargs):
    result = func(*args, **kwargs)
    assert type(result) == dict
    return result
  return wrapper

@returns_dict
def foo(value):
  return value

try:
  print(foo([1,2,3]))
except AssertionError:
  print('foo() did not return a dict!')

foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!


In [18]:
def returns(return_type):
  # Write a decorator that raises an AssertionError if the
  # decorated function returns a value that is not return_type
  def decorator(func):
    def wrapper(*args, **kwargs):
      result = func(*args, **kwargs)
      assert type(result) == return_type
      return result
    return wrapper
  return decorator
  
@returns(dict)
def foo(value):
  return value

try:
  print(foo([1,2,3]))
except AssertionError:
  print('foo() did not return a dict!')

foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!
foo() did not return a dict!


You did it! We took the training wheels off on this exercise, and you still did a great job. You know how to write your own decorators now, but even more importantly, you know why they work the way they do.